In [ ]:
# pip install tensorflow==2.9.0
# pip install pandas
# pip install -U scikit-learn
# pip install pyyaml h5py

# Installation des dépendances requises pour le projet.

In [ ]:
print("TensorFlow version: ", tf.__version__)

# Afficher la version de TensorFlow
Cette ligne de code affiche la version actuelle de TensorFlow qui est utilisée dans l'environnement Python. Cela peut être utile pour confirmer que la bonne version de TensorFlow est installée ou pour s'assurer que le code fonctionnera sur d'autres machines avec la même version de TensorFlow.

In [2]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

ModuleNotFoundError: No module named 'cv2'

# Importation des bibliothèques et des outils nécessaires

- Dans cette section, nous importons des bibliothèques système telles que os, time, shutil, pathlib et itertools.

- Dans cette section, nous importons des bibliothèques pour le traitement de données telles que cv2, numpy, pandas, seaborn et matplotlib.pyplot. Nous utilisons également des outils de scikit-learn tels que train_test_split, confusion_matrix et classification_report.

- Dans cette section, nous importons des bibliothèques de Deep Learning telles que tensorflow et keras. Nous utilisons également des couches spécifiques de Keras telles que Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout et BatchNormalization. Nous utilisons également des optimiseurs tels qu'Adam et Adamax.

- Dans cette section, nous ignorons les avertissements pour rendre la sortie du code plus propre et lisible.

- Enfin, nous affichons un message de confirmation pour indiquer que toutes les bibliothèques ont été importées avec succès.

In [ ]:
# Generate data paths with labels
data_dir = 'chest_xray/train'
filepaths = []
labels = []

folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis= 1)

# Générer des chemins de données avec des étiquettes
Ce code lit les données d'un répertoire spécifié (dans ce cas, le dossier `chest_xray/train`), puis crée une liste de chemins de fichiers et d'étiquettes correspondantes. Il parcourt d'abord tous les dossiers du répertoire et ajoute tous les fichiers de chaque dossier à la liste de chemins de fichiers. Il utilise ensuite le nom du dossier comme étiquette pour chaque fichier correspondant. 

Enfin, les chemins de fichiers et les étiquettes sont combinés dans un dataframe pandas pour faciliter la manipulation des données.

In [ ]:
# show dataframe
df

# Afficher le dataframe
Cette partie du code affiche simplement le dataframe `df`, qui contient les chemins d'accès des images et leurs étiquettes correspondantes. Il peut être utile d'examiner ce dataframe pour s'assurer que les données sont correctement formatées et étiquetées. 

Pour afficher le dataframe, il suffit d'exécuter ce bloc de code en sélectionnant la cellule et en appuyant sur Shift+Enter ou en cliquant sur le bouton "Exécuter" dans la barre d'outils. 

In [ ]:
# train dataframe
train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 123)

# valid and test dataframe
valid_df, test_df = train_test_split(dummy_df,  train_size= 0.6, shuffle= True, random_state= 123)

# Diviser les données en ensembles d'entraînement, de validation et de test
Cette partie du code utilise la fonction `train_test_split` de la bibliothèque scikit-learn pour diviser le dataframe `df` en ensembles d'entraînement, de validation et de test. 

Il divise d'abord `df` en deux ensembles: l'ensemble d'entraînement et un ensemble qui contient les données restantes. Ensuite, il divise l'ensemble de données restantes en deux ensembles: l'ensemble de validation et l'ensemble de test.

La taille de chaque ensemble est contrôlée par les paramètres `train_size` et `test_size` de la fonction `train_test_split`, qui sont respectivement définis à 0,8 et 0,6 dans ce cas. Le paramètre `shuffle` contrôle si les données sont mélangées avant la division, et `random_state` fixe la graine pour la génération de nombres aléatoires, garantissant ainsi que la même division sera obtenue à chaque exécution.

In [ ]:
# crobed image size
batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

# Recommended : use custom function for test data batch size, else we can use normal batch size.
ts_length = len(test_df)
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

# This function which will be used in image data generator for data augmentation, it just take the image and return it again.
def scalar(img):
    return img

tr_gen = ImageDataGenerator(preprocessing_function= scalar)
ts_gen = ImageDataGenerator(preprocessing_function= scalar)

train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

# Note: we will use custom test_batch_size, and make shuffle= false
test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= test_batch_size)

# Création de générateurs de données d'images pour l'entraînement, la validation et le test.

- Dans un premier temps, nous définition des tailles d'images.

- Détermination de la taille de lot recommandée pour les données de test, à l'aide d'une fonction personnalisée.
Cette fonction calcule la taille optimale pour chaque lot de données, en divisant le nombre d'échantillons de test
par un nombre entier jusqu'à 80. La taille maximale est choisie pour minimiser les calculs tout en utilisant
la mémoire disponible.

- Définition d'une fonction qui sera utilisée pour effectuer l'augmentation des données de l'ensemble d'entraînement.
Cette fonction prend simplement l'image en entrée et la renvoie telle quelle.

- Création des générateurs d'images pour l'ensemble d'entraînement et l'ensemble de validation.

- Création du générateur d'images pour l'ensemble de test.
Note: nous allons utiliser une taille de lot personnalisée pour les données de test, et shuffle=False pour ne pas
perturber l'ordre des images dans les prédictions.

In [ ]:
g_dict = train_gen.class_indices      # defines dictionary {'class': index}
classes = list(g_dict.keys())       # defines list of dictionary's kays (classes), classes names : string
images, labels = next(train_gen)      # get a batch size samples from the generator

plt.figure(figsize= (20, 20))

for i in range(16):
    plt.subplot(4, 4, i + 1)
    image = images[i] / 255       # scales data to range (0 - 255)
    plt.imshow(image)
    index = np.argmax(labels[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'blue', fontsize= 12)
    plt.axis('off')
plt.show()

# Affichage d'un lot d'images avec leurs étiquettes

Ce code affiche un lot de 16 images à partir d'un générateur de données train_gen. Il utilise un dictionnaire class_indices pour mapper chaque nom de classe à son indice correspondant. Ensuite, il récupère une liste de noms de classe à partir des clés du dictionnaire.

Le code utilise ensuite la fonction next() pour extraire un lot de données et d'étiquettes à partir du générateur. Chaque image est normalisée en divisant ses valeurs de pixel par 255 pour les mettre à l'échelle dans la plage (0 - 1).

Enfin, le code affiche chaque image avec son étiquette correspondante dans un sous-ensemble de la figure matplotlib, en utilisant le nom de la classe pour étiqueter chaque image.

In [ ]:
# Create Model Structure
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer# Create Model Structure
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer

# create pre-trained model (you can built on pretrained model such as :  efficientnet, VGG , Resnet )
# we will use efficientnetb3 from EfficientNet family.
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')
# base_model.trainable = False

# Define a simple sequential model
def create_model():
    model = Sequential([
        base_model,
        BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
        Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                    bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
        Dropout(rate= 0.45, seed= 123),
        Dense(class_count, activation= 'softmax')
    ])

    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

    return model

model = create_model()
model.summary()

# Création de la structure du modèle et définition de sa compilation

La fonction create_model() crée le modèle en utilisant les couches mentionnées ci-dessus et définit les paramètres de compilation. Le modèle est ensuite compilé avec l'optimiseur Adamax, une fonction de coût categorical_crossentropy et une métrique d'évaluation de précision.

La variable class_count est définie à partir de la liste des indices de classes fournies par le générateur de données train_gen pour déterminer le nombre de classes dans la couche de sortie. Le modèle est ensuite instancié en appelant create_model() et son architecture est affichée en utilisant la méthode summary().

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

epochs = 10 # number of all epochs in training

history = model.fit(x= train_gen,
                    epochs= epochs,
                    verbose= 1,
                    validation_data= valid_gen, 
                    validation_steps= None,
                    shuffle= False,
                    callbacks=[cp_callback])  # Pass callback to training  

# Enregistrer les poids du modèle à intervalles réguliers pendant l'entraînement

Ce code utilise la fonction tf.keras.callbacks.ModelCheckpoint pour enregistrer les poids du modèle à intervalles réguliers pendant l'entraînement. Il prend en entrée plusieurs paramètres pour personnaliser le comportement du rappel de contrôle, tels que le chemin d'accès au fichier de point de contrôle, la fréquence d'enregistrement des points de contrôle, etc.

Le code crée également une instance du rappel de contrôle et la passe au modèle lors de l'appel à la méthode fit() pour l'entraînement. De cette façon, le modèle enregistrera automatiquement les poids à chaque intervalle spécifié par le rappel de contrôle.

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Enregistrer les poids du modèle

Ce code utilise la méthode save_weights() pour enregistrer les poids du modèle dans un fichier sur le disque. Le chemin d'accès et le nom du fichier de point de contrôle sont spécifiés dans la chaîne passée à la méthode save_weights().

In [ ]:
# Create a new model instance
model = create_model()

latest = tf.train.latest_checkpoint(checkpoint_dir)

# end the training


# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(train_gen, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))



# Charger des poids précédemment sauvegardés

Ce code crée une nouvelle instance de modèle à partir de la fonction create_model() et utilise tf.train.latest_checkpoint() pour trouver le dernier point de contrôle sauvegardé dans le répertoire spécifié. Ensuite, le modèle charge les poids précédemment sauvegardés à l'aide de model.load_weights().

In [ ]:
model.save('saved_model/my_model1') # downgrade to 2.9.1 tf version

Enfin, ce code enregistre le modèle actuel en utilisant la fonction model.save() avec un nom de fichier spécifié. Dans cet exemple, le nom de fichier est 'saved_model/my_model1'. Cela permet de sauvegarder le modèle et ses poids dans un format standard pour les modèles TensorFlow.